<a href="https://colab.research.google.com/github/mmtaha/Projeto_busca_de_emprego/blob/main/teste_busca_emprego_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Etapa 1: Processamento do Currículo e Extração de Palavras-chave

import os
import docx
import spacy
import pytextrank
import yake
from rake_nltk import Rake
from nltk.corpus import stopwords
from nltk import download
import re

# Baixar recursos necessários
import nltk
nltk.download("punkt")
nltk.download("stopwords")

# Diretórios fixos
DIR_CURR = "/content/curriculo"
DIR_SAMPLE = "/content/sample_data"
DIR_PROC = "/content/Processamento"
DIR_OUT = "/content/Output"

for path in [DIR_CURR, DIR_SAMPLE, DIR_PROC, DIR_OUT]:
    os.makedirs(path, exist_ok=True)

# Credenciais
CRED = {
    #'linkedin': {'email': '********', 'senha': '******'},
    #'glassdoor': {'email': '***********', 'senha': '**********'},
    #'vagas': {'usuario': '*****', 'senha': '************'},
    #'indeed': {'email': '***********', 'senha': '************'}
}

# Modelos spaCy e extratores
nlpt = spacy.load("pt_core_news_sm")
nlpt.add_pipe("textrank")

# Extratores
yake_extractor = yake.KeywordExtractor(lan="pt", top=100, n=1)
rake = Rake(language="portuguese", stopwords=stopwords.words("portuguese"))

# POS tags a filtrar
FILTER_POS = {'PRON', 'DET', 'ADP', 'AUX', 'CCONJ', 'SCONJ', 'PART', 'INTJ', 'PUNCT', 'SPACE', 'NUM'}

# Utilitários

def extrair_texto_docx(file_path):
    doc = docx.Document(file_path)
    return '\n'.join([p.text for p in doc.paragraphs])

def salvar_texto(texto, nome_arquivo):
    with open(os.path.join(DIR_PROC, nome_arquivo), 'w', encoding='utf-8') as f:
        f.write(texto)

def limpar_com_filtragem(texto, modelo_spacy):
    doc = modelo_spacy(texto)
    tokens_filtrados = [token.lemma_.lower() for token in doc
                        if token.pos_ not in FILTER_POS and not token.is_stop and token.is_alpha]
    return ' '.join(tokens_filtrados)

def extrair_keywords(texto, lang_model):
    texto_limpo = limpar_com_filtragem(texto, lang_model)

    # YAKE
    keywords_yake = [kw for kw, score in yake_extractor.extract_keywords(texto_limpo)]

    # RAKE
    rake.extract_keywords_from_text(texto_limpo)
    keywords_rake = rake.get_ranked_phrases()

    # TextRank
    doc = lang_model(texto_limpo)
    keywords_textrank = [phrase.text.lower() for phrase in doc._.phrases]

    return {
        'yake': keywords_yake[:20],
        'rake': keywords_rake[:20],
        'textrank': keywords_textrank[:20]
    }

def processar_curriculos():
    for arquivo in os.listdir(DIR_CURR):
        if arquivo.endswith(".docx"):
            caminho_arquivo = os.path.join(DIR_CURR, arquivo)
            texto_extraido = extrair_texto_docx(caminho_arquivo)

            # Salvar texto bruto
            salvar_texto(texto_extraido, f"{arquivo.replace('.docx', '')}_processado.txt")

            # Extrair palavras-chave
            resultados = extrair_keywords(texto_extraido, nlpt)

            # Salvar resultados
            for metodo, palavras in resultados.items():
                with open(os.path.join(DIR_OUT, f"{arquivo.replace('.docx', '')}_{metodo}.txt"), 'w', encoding='utf-8') as f:
                    f.write('\n'.join(palavras))

# Executar a primeira etapa
processar_curriculos()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import os
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from docx import Document

# Diretórios
DIR_PROC = "/content/Processamento"
DIR_OUT = "/content/Output"

# POS a serem filtradas
FILTER_POS = {'PRON', 'DET', 'ADP', 'AUX', 'CCONJ', 'SCONJ', 'PART', 'INTJ', 'PUNCT', 'SPACE', 'NUM'}

# Carregando modelo spaCy
nlp = spacy.load("pt_core_news_sm")

def limpar_texto(texto):
    doc = nlp(texto.lower())
    return " ".join([token.lemma_ for token in doc if token.pos_ not in FILTER_POS and not token.is_stop and not token.is_punct])

# Função para ler arquivos DOCX do diretório
def ler_arquivos_docx(diretorio):
    textos = []
    for nome_arquivo in os.listdir(diretorio):
        if nome_arquivo.endswith(".docx"):
            path = os.path.join(diretorio, nome_arquivo)
            doc = Document(path)
            full_text = " ".join([p.text for p in doc.paragraphs])
            textos.append((nome_arquivo, full_text))
    return textos

# Carregar currículos e vagas processadas
curriculos = ler_arquivos_docx(DIR_PROC)
vagas = ler_arquivos_docx(DIR_OUT)

# Pré-processar os textos
curriculos_proc = [(nome, limpar_texto(texto)) for nome, texto in curriculos]
vagas_proc = [(nome, limpar_texto(texto)) for nome, texto in vagas]

# Vetorizar
all_docs = [texto for _, texto in curriculos_proc + vagas_proc]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(all_docs)

# Separar vetores
vetores_curriculos = X[:len(curriculos_proc)]
vetores_vagas = X[len(curriculos_proc):]

# Calcular similaridade
resultado_analise = []
for i, (nome_curriculo, _) in enumerate(curriculos_proc):
    for j, (nome_vaga, texto_vaga) in enumerate(vagas_proc):
        sim = cosine_similarity(vetores_curriculos[i], vetores_vagas[j])[0][0]
        resultado_analise.append({
            "curriculo": nome_curriculo,
            "vaga": nome_vaga,
            "similaridade": round(sim * 100, 2)
        })

# Exportar resultados
output_doc = Document()
output_doc.add_heading("Análise de Aderência entre Currículos e Vagas", level=1)

for resultado in sorted(resultado_analise, key=lambda x: x['similaridade'], reverse=True):
    output_doc.add_paragraph(
        f"Currículo: {resultado['curriculo']} \n"
        f"Vaga: {resultado['vaga']} \n"
        f"Aderência: {resultado['similaridade']}%"
    )
    output_doc.add_paragraph("\n")

# Salvar resultado
saida_final = os.path.join(DIR_PROC, "resultado_aderencia.docx")
output_doc.save(saida_final)
print(f"Análise salva em: {saida_final}")



In [ ]:
import os
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from docx import Document

# Diretórios
DIR_PROC = "/content/Processamento"
DIR_OUT = "/content/Output"

# POS a serem filtradas
FILTER_POS = {'PRON', 'DET', 'ADP', 'AUX', 'CCONJ', 'SCONJ', 'PART', 'INTJ', 'PUNCT', 'SPACE', 'NUM'}

# Carregando modelo spaCy
nlp = spacy.load("pt_core_news_sm")

def limpar_texto(texto):
    doc = nlp(texto.lower())
    return " ".join([token.lemma_ for token in doc if token.pos_ not in FILTER_POS and not token.is_stop and not token.is_punct])

# Função para ler arquivos DOCX do diretório
def ler_arquivos_docx(diretorio):
    textos = []
    for nome_arquivo in os.listdir(diretorio):
        if nome_arquivo.endswith(".docx"):
            path = os.path.join(diretorio, nome_arquivo)
            doc = Document(path)
            full_text = " ".join([p.text for p in doc.paragraphs])
            textos.append((nome_arquivo, full_text))
    return textos

# Carregar currículos e vagas processadas
curriculos = ler_arquivos_docx(DIR_PROC)
vagas = ler_arquivos_docx(DIR_OUT)

# Pré-processar os textos
curriculos_proc = [(nome, limpar_texto(texto)) for nome, texto in curriculos]
vagas_proc = [(nome, limpar_texto(texto)) for nome, texto in vagas]

# Vetorizar
all_docs = [texto for _, texto in curriculos_proc + vagas_proc]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(all_docs)

# Separar vetores
vetores_curriculos = X[:len(curriculos_proc)]
vetores_vagas = X[len(curriculos_proc):]

# Calcular similaridade
resultado_analise = []
for i, (nome_curriculo, _) in enumerate(curriculos_proc):
    for j, (nome_vaga, texto_vaga) in enumerate(vagas_proc):
        sim = cosine_similarity(vetores_curriculos[i], vetores_vagas[j])[0][0]
        resultado_analise.append({
            "curriculo": nome_curriculo,
            "vaga": nome_vaga,
            "similaridade": round(sim * 100, 2)
        })

# Exportar resultados
output_doc = Document()
output_doc.add_heading("Análise de Aderência entre Currículos e Vagas", level=1)

for resultado in sorted(resultado_analise, key=lambda x: x['similaridade'], reverse=True):
    output_doc.add_paragraph(
        f"Currículo: {resultado['curriculo']} \n"
        f"Vaga: {resultado['vaga']} \n"
        f"Aderência: {resultado['similaridade']}%"
    )
    output_doc.add_paragraph("\n")

# Salvar resultado
saida_final = os.path.join(DIR_PROC, "resultado_aderencia.docx")
output_doc.save(saida_final)
print(f"Análise salva em: {saida_final}")
